In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio

NUM_COLUMNS=19
BATCH_SIZE=64
MAX_EPOCHS=10

In [ ]:
def decode_kafka_item(item):
    vect = tf.io.decode_csv(item.message, [[0.0] for i in range(NUM_COLUMNS)])
    t = [vect.pop(1)]
    return (vect, t)

train_ds = tfio.IODataset.from_kafka("tf.public.weather_train", partition=0, offset=0, servers="kafka:9092")
train_ds = train_ds.map(decode_kafka_item)
train_ds = train_ds.batch(BATCH_SIZE)

In [ ]:
linear = tf.keras.Sequential([
   tf.keras.layers.Dense(units=1)
])

linear.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
history = linear.fit(train_ds, epochs=MAX_EPOCHS)

In [ ]:
def decode_kafka_test_item(item_message, item_key):
    vect = tf.io.decode_csv(item_message, [[0.0] for i in range(NUM_COLUMNS)])
    t = [vect.pop(1)]
    return (vect, t)

test_ds = tfio.experimental.streaming.KafkaGroupIODataset(
    topics=["tf.weather.weather_test"],
    group_id="testcg2",
    servers="kafka:9092",
    stream_timeout=10000,
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
        "auto.offset.reset=earliest"
    ],
)

In [ ]:
test_ds = test_ds.map(decode_kafka_test_item)
test_ds = test_ds.batch(BATCH_SIZE)

res = linear.evaluate(test_ds)
print("test loss, test acc:", res)